In [4]:
import numpy as np
from chess import pgn
import chess.engine

In [5]:
BATCHES = 50
BATCH_SIZE = 20000

In [6]:
def make_bitboard(board):
    bitboard = np.empty(shape=(12, 8, 8))

    for piece in chess.PIECE_TYPES:
        index = piece - 1 # arrays are zero indexed
        white_layer = np.reshape(board.pieces(piece, chess.BLACK).tolist(), (8, 8)).astype(int)
        bitboard[index] = white_layer
        black_layer = np.reshape(board.pieces(piece, chess.WHITE).tolist(), (8, 8)).astype(int)
        bitboard[index + len(chess.PIECE_TYPES)] = black_layer # shift index up amount of pieces for black

    return bitboard

In [7]:
stockfish = chess.engine.SimpleEngine.popen_uci('/opt/homebrew/Cellar/stockfish/15/bin/stockfish')

def generate_dataset(batches, batch_size):
    month = 11
    batch_counter = 0
    while batch_counter < batches and month > 0:
        with open(f'data/2022-{month:02d}.bare.pgn') as pgn_file:
            game = pgn.read_game(pgn_file)
            bitboards = np.empty(shape=(batch_size,12,8,8), dtype=np.uint8)
            scores = np.empty(shape=(batch_size), dtype=np.int16)
            game_counter = 1
            batch_size_counter = 0
            while game is not None and batch_counter < batches:
                board = game.board()
                move_counter = 0
                for move in game.mainline_moves():
                    if batch_counter >= batches:
                        break
                    move_counter += 1
                    board.push(move)
                    # if checkmate in some moves found assign large score (i.e. 30000)
                    scores[batch_size_counter] = stockfish.analyse(board, chess.engine.Limit(depth=10))['score'].white().score(mate_score=30000)
                    bitboards[batch_size_counter] = make_bitboard(board)
                    batch_size_counter += 1
                    if batch_size_counter >= batch_size:
                        batch_counter += 1
                        batch_size_counter = 0
                        np.savez(f'data/batch_{batch_counter:02d}.npz', bitboards=bitboards, scores=scores)
                        print(f'Batch {batch_counter:02d} completed at move {move_counter} of game {game_counter} of month {month}')
                game = pgn.read_game(pgn_file)
                game_counter += 1
        month -= 1

generate_dataset(BATCHES, BATCH_SIZE)

Batch 01 completed at move 145 of game 164 of month 11
Batch 02 completed at move 23 of game 333 of month 11
Batch 03 completed at move 14 of game 493 of month 11
Batch 04 completed at move 90 of game 647 of month 11
Batch 05 completed at move 13 of game 814 of month 11
Batch 06 completed at move 93 of game 975 of month 11
Batch 07 completed at move 120 of game 1134 of month 11
Batch 08 completed at move 23 of game 1284 of month 11
Batch 09 completed at move 30 of game 1437 of month 11
Batch 10 completed at move 70 of game 1591 of month 11
Batch 11 completed at move 87 of game 1751 of month 11
Batch 12 completed at move 20 of game 1906 of month 11
Batch 13 completed at move 68 of game 2072 of month 11
Batch 14 completed at move 12 of game 2234 of month 11
Batch 15 completed at move 165 of game 2401 of month 11
Batch 16 completed at move 56 of game 2565 of month 11
Batch 17 completed at move 124 of game 2726 of month 11
Batch 18 completed at move 39 of game 2892 of month 11
Batch 19 com

In [47]:
def generate_dataset(batches, batch_size, start_batch=1, start_month=11, start_game=1, start_move=1, stockfish_depth=10):
    stockfish = chess.engine.SimpleEngine.popen_uci('/opt/homebrew/Cellar/stockfish/15/bin/stockfish')

    month = start_month
    batch_counter = start_batch
    while batch_counter < (start_batch + batches) and month > 0:
        with open(f'data/2022-{month:02d}.bare.pgn') as pgn_file:
            game = pgn.read_game(pgn_file)
            game_counter = 1

            while game_counter < start_game:
                game = pgn.read_game(pgn_file)
                game_counter += 1
            bitboards = np.empty(shape=(batch_size,12,8,8), dtype=np.uint8)
            scores = np.empty(shape=(batch_size), dtype=np.int16)
            batch_size_counter = 0

            while game is not None and batch_counter < (start_batch + batches):
                board = game.board()
                move_counter = 1

                for move in game.mainline_moves():
                    board.push(move)

                    if move_counter < start_move:
                        move_counter += 1
                        continue

                    # if checkmate in some moves found assign large score (i.e. 30000)
                    scores[batch_size_counter] = stockfish.analyse(board, chess.engine.Limit(depth=stockfish_depth))['score'].white().score(mate_score=30000)
                    bitboards[batch_size_counter] = make_bitboard(board)
                    batch_size_counter += 1
                    if batch_size_counter >= batch_size:
                        np.savez(f'data/batches/depth/{stockfish_depth:02d}/batch_{batch_counter:02d}.npz', bitboards=bitboards, scores=scores)
                        print(f'Batch {batch_counter:02d} completed at move {move_counter} of game {game_counter} of month {month}')
                        batch_counter += 1
                        batch_size_counter = 0

                    
                    if batch_counter >= (start_batch + batches):
                        break
                    move_counter += 1

                game = pgn.read_game(pgn_file)
                game_counter += 1

            start_game = 1
            start_move = 1
        month -= 1

In [40]:
#generate_dataset(batches=50, batch_size=20000, start_batch=51, start_month=10, start_game=961, start_move=85)
generate_dataset(batches=50, batch_size=20000, stockfish_depth=5)

Batch 01 completed at move 145 of game 164 of month 11
Batch 02 completed at move 23 of game 333 of month 11
Batch 03 completed at move 14 of game 493 of month 11
Batch 04 completed at move 90 of game 647 of month 11
Batch 05 completed at move 13 of game 814 of month 11
Batch 06 completed at move 93 of game 975 of month 11
Batch 07 completed at move 120 of game 1134 of month 11
Batch 08 completed at move 23 of game 1284 of month 11
Batch 09 completed at move 30 of game 1437 of month 11
Batch 10 completed at move 70 of game 1591 of month 11
Batch 11 completed at move 87 of game 1751 of month 11
Batch 12 completed at move 20 of game 1906 of month 11
Batch 13 completed at move 68 of game 2072 of month 11
Batch 14 completed at move 12 of game 2234 of month 11
Batch 15 completed at move 165 of game 2401 of month 11
Batch 16 completed at move 56 of game 2565 of month 11
Batch 17 completed at move 124 of game 2726 of month 11
Batch 18 completed at move 39 of game 2892 of month 11
Batch 19 com

In [48]:
generate_dataset(batches=50, batch_size=20000, stockfish_depth=0)

Batch 01 completed at move 145 of game 164 of month 11
Batch 02 completed at move 23 of game 333 of month 11
Batch 03 completed at move 14 of game 493 of month 11
Batch 04 completed at move 90 of game 647 of month 11
Batch 05 completed at move 13 of game 814 of month 11
Batch 06 completed at move 93 of game 975 of month 11
Batch 07 completed at move 120 of game 1134 of month 11
Batch 08 completed at move 23 of game 1284 of month 11
Batch 09 completed at move 30 of game 1437 of month 11
Batch 10 completed at move 70 of game 1591 of month 11
Batch 11 completed at move 87 of game 1751 of month 11
Batch 12 completed at move 20 of game 1906 of month 11
Batch 13 completed at move 68 of game 2072 of month 11
Batch 14 completed at move 12 of game 2234 of month 11
Batch 15 completed at move 165 of game 2401 of month 11
Batch 16 completed at move 56 of game 2565 of month 11
Batch 17 completed at move 124 of game 2726 of month 11
Batch 18 completed at move 39 of game 2892 of month 11
Batch 19 com

In [50]:
generate_dataset(batches=50, batch_size=20000, stockfish_depth=1)

Batch 01 completed at move 145 of game 164 of month 11
Batch 02 completed at move 23 of game 333 of month 11
Batch 03 completed at move 14 of game 493 of month 11
Batch 04 completed at move 90 of game 647 of month 11
Batch 05 completed at move 13 of game 814 of month 11
Batch 06 completed at move 93 of game 975 of month 11
Batch 07 completed at move 120 of game 1134 of month 11
Batch 08 completed at move 23 of game 1284 of month 11
Batch 09 completed at move 30 of game 1437 of month 11
Batch 10 completed at move 70 of game 1591 of month 11
Batch 11 completed at move 87 of game 1751 of month 11
Batch 12 completed at move 20 of game 1906 of month 11
Batch 13 completed at move 68 of game 2072 of month 11
Batch 14 completed at move 12 of game 2234 of month 11
Batch 15 completed at move 165 of game 2401 of month 11
Batch 16 completed at move 56 of game 2565 of month 11
Batch 17 completed at move 124 of game 2726 of month 11
Batch 18 completed at move 39 of game 2892 of month 11
Batch 19 com

In [52]:
generate_dataset(batches=50, batch_size=20000, stockfish_depth=2)

Batch 01 completed at move 145 of game 164 of month 11
Batch 02 completed at move 23 of game 333 of month 11
Batch 03 completed at move 14 of game 493 of month 11
Batch 04 completed at move 90 of game 647 of month 11
Batch 05 completed at move 13 of game 814 of month 11
Batch 06 completed at move 93 of game 975 of month 11
Batch 07 completed at move 120 of game 1134 of month 11
Batch 08 completed at move 23 of game 1284 of month 11
Batch 09 completed at move 30 of game 1437 of month 11
Batch 10 completed at move 70 of game 1591 of month 11
Batch 11 completed at move 87 of game 1751 of month 11
Batch 12 completed at move 20 of game 1906 of month 11
Batch 13 completed at move 68 of game 2072 of month 11
Batch 14 completed at move 12 of game 2234 of month 11
Batch 15 completed at move 165 of game 2401 of month 11
Batch 16 completed at move 56 of game 2565 of month 11
Batch 17 completed at move 124 of game 2726 of month 11
Batch 18 completed at move 39 of game 2892 of month 11
Batch 19 com

In [53]:
generate_dataset(batches=50, batch_size=20000, start_batch=51, start_month=10, start_game=961, start_move=85, stockfish_depth=1)

Batch 51 completed at move 256 of game 1454 of month 10
Batch 52 completed at move 142 of game 1939 of month 10
Batch 53 completed at move 96 of game 2416 of month 10
Batch 54 completed at move 142 of game 2849 of month 10
Batch 55 completed at move 88 of game 3290 of month 10
Batch 56 completed at move 107 of game 3728 of month 10
Batch 57 completed at move 98 of game 4187 of month 10
Batch 58 completed at move 85 of game 4650 of month 10
Batch 59 completed at move 123 of game 5111 of month 10
Batch 60 completed at move 99 of game 5486 of month 10
Batch 61 completed at move 91 of game 5981 of month 10
Batch 62 completed at move 206 of game 6384 of month 10
Batch 63 completed at move 108 of game 6810 of month 10
Batch 64 completed at move 94 of game 7282 of month 10
Batch 65 completed at move 114 of game 7734 of month 10
Batch 66 completed at move 123 of game 8144 of month 10
Batch 67 completed at move 186 of game 8568 of month 10
Batch 68 completed at move 104 of game 8993 of month 10

In [54]:
generate_dataset(batches=400, batch_size=20000, start_batch=101, start_month=10, start_game=961, start_move=85, stockfish_depth=1)

Batch 101 completed at move 256 of game 1454 of month 10
Batch 102 completed at move 142 of game 1939 of month 10
Batch 103 completed at move 96 of game 2416 of month 10
Batch 104 completed at move 142 of game 2849 of month 10
Batch 105 completed at move 88 of game 3290 of month 10
Batch 106 completed at move 107 of game 3728 of month 10
Batch 107 completed at move 98 of game 4187 of month 10
Batch 108 completed at move 85 of game 4650 of month 10
Batch 109 completed at move 123 of game 5111 of month 10
Batch 110 completed at move 99 of game 5486 of month 10
Batch 111 completed at move 91 of game 5981 of month 10
Batch 112 completed at move 206 of game 6384 of month 10
Batch 113 completed at move 108 of game 6810 of month 10
Batch 114 completed at move 94 of game 7282 of month 10
Batch 115 completed at move 114 of game 7734 of month 10
Batch 116 completed at move 123 of game 8144 of month 10
Batch 117 completed at move 186 of game 8568 of month 10
Batch 118 completed at move 104 of gam